# Wind Turbine Detection

# Data Description

The dataset was taken from: https://www.kaggle.com/datasets/saurabhshahane/wind-turbine-obj-detection

The dataset contains a set of overhead images of wind turbines with corresponding YOLO formatted labels for object detection. These labels contain the class, x and y coordinates and the height and width of the bounding boxes for each wind turbine in the corresponding image.

# Importing necessary libraries & loading data

In [1]:
! pip install pytorch-lightning

     |████████████████████████████████| 584 kB 4.2 MB/s 
     |████████████████████████████████| 136 kB 51.2 MB/s 
     |████████████████████████████████| 409 kB 47.2 MB/s 
     |████████████████████████████████| 596 kB 38.7 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 144 kB 53.2 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 43.7 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.3 MB/s 
     |████████████████████████████████| 948 kB 28.4 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=6e77a3b3af50f0480e6a670870c0c2ae154b10ae6591a1f9ce47aa1bc3779b53
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [3]:
! pip install effdet

     |████████████████████████████████| 112 kB 4.4 MB/s 
     |████████████████████████████████| 74 kB 2.4 MB/s 
     |████████████████████████████████| 431 kB 39.2 MB/s 
     |████████████████████████████████| 112 kB 47.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=809ab0fd46d92506e4a89de284296f6f8be2823a892f4b03e356b050593393f9
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime


In [4]:
! pip install pybboxes

In [5]:
! pip install kaggle

In [6]:
import os
from tqdm.autonotebook import tqdm, trange
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import copy
import time
from torch.utils.data import Dataset, DataLoader
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import albumentations.augmentations.functional as F
from pathlib import Path
import PIL
from effdet.config.model_config import efficientdet_model_param_dict
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from effdet.config.model_config import efficientdet_model_param_dict
from pytorch_lightning import LightningDataModule
from pytorch_lightning import LightningModule
#from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning import Trainer

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [7]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
! kaggle datasets download -d saurabhshahane/wind-turbine-obj-detection

 97% 233M/239M [00:10<00:00, 20.6MB/s]
100% 239M/239M [00:10<00:00, 24.0MB/s]


In [11]:
! unzip wind-turbine-obj-detection.zip

Archive:  wind-turbine-obj-detection.zip
  inflating: images/images/forest-wind-crop-0.jpg  
  inflating: images/images/forest-wind-crop-1.jpg  
  inflating: images/images/forest-wind-crop-10.jpg  
  inflating: images/images/forest-wind-crop-11.jpg  
  inflating: images/images/forest-wind-crop-12.jpg  
  inflating: images/images/forest-wind-crop-13.jpg  
  inflating: images/images/forest-wind-crop-14.jpg  
  inflating: images/images/forest-wind-crop-15.jpg  
  inflating: images/images/forest-wind-crop-16.jpg  
  inflating: images/images/forest-wind-crop-17.jpg  
  inflating: images/images/forest-wind-crop-18.jpg  
  inflating: images/images/forest-wind-crop-19.jpg  
  inflating: images/images/forest-wind-crop-2.jpg  
  inflating: images/images/forest-wind-crop-20.jpg  
  inflating: images/images/forest-wind-crop-21.jpg  
  inflating: images/images/forest-wind-crop-22.jpg  
  inflating: images/images/forest-wind-crop-23.jpg  
  inflating: images/images/forest-wind-crop-3.jpg  
  inflati